### Removed terminal sugars based on SRU

In [ ]:
import pandas as pd
from rdkit import rdBase, Chem

# Read the results of deglycosylation
df = pd.read_csv("deglycosylation_results.txt")
str1 = 'Nr;ID;originalMoleculeSMILES;deglycosylatedMoleculeSMILES;hadOrHasSugars;SugarMoietySMILES'
list1 = str1.split(';')
df1 = pd.DataFrame(columns=list1[1:])
def splite(x):
    p = x.split(";")[1:]
    return p
q1 = df["Nr;ID;originalMoleculeSMILES;deglycosylatedMoleculeSMILES;hadOrHasSugars;SugarMoietySMILES"].map(splite)
ID = []
originalMoleculeSMILES = []
deglycosylatedMoleculeSMILES = []
hadOrHasSugars = []
SugarMoietySMILES = []
for i in range(len(q1)):
    ID.append(q1[i][0])
    originalMoleculeSMILES.append(q1[i][1])
    deglycosylatedMoleculeSMILES.append(q1[i][2])
    hadOrHasSugars.append(q1[i][3])
    if len(q1[i])>=5:
        SugarMoietySMILES.append(q1[i][4:])
    else:
        SugarMoietySMILES.append([])
df1["ID"] = ID
df1["originalMoleculeSMILES"] = originalMoleculeSMILES
df1["deglycosylatedMoleculeSMILES"] = deglycosylatedMoleculeSMILES
df1["hadOrHasSugars"] = hadOrHasSugars
df1["SugarMoietySMILES"] = SugarMoietySMILES

# Standardized the structure of compounds based on RDKit
originalMoleculeSMILES = []
deglycosylatedMoleculeSMILES = []
ID_list = []
for i in range(len(df1)):
    origi = df1['originalMoleculeSMILES'][i]
    degly = df1['deglycosylatedMoleculeSMILES'][i]
    try:
        ori_smi = Chem.MolToSmiles(Chem.MolFromSmiles(origi))
        originalMoleculeSMILES.append(ori_smi) 
        if degly == '[empty]':
            deglycosylatedMoleculeSMILES.append(degly)
        else:
            degly_smi = Chem.MolToSmiles(Chem.MolFromSmiles(degly))
            deglycosylatedMoleculeSMILES.append(degly_smi)
        ID_list.append(df1['ID'][i])
    except:
        pass
df1 = df1[df1.ID.isin(ID_list)]
df1['originalMoleculeSMILES'] = originalMoleculeSMILES
df1['deglycosylatedMoleculeSMILES'] = deglycosylatedMoleculeSMILES 
df1.to_csv("deglycosylation_results.csv",index=False)

# Statistics for the glycosylation ratio of NPs
df_HasSugar=df1[df1['hadOrHasSugars']=='true']
df_HasSugar.to_csv('HasSugar.csv',index=False)
df_NoSugar=df1[df1['hadOrHasSugars']=='false']
df_NoSugar.to_csv('NoSugar.csv',index=False)
df_Sugar_Percentage=len(df_HasSugar)/(len(df_HasSugar)+len(df_NoSugar))*100
Glycosides=[]
Glycosides.append((len(df_HasSugar),len(df_NoSugar),df_Sugar_Percentage))
Glycosides=pd.DataFrame(Glycosides,columns=['HasSugar','NoSugar','Percentage(%)'])
Glycosides['Percentage(%)']=round(Glycosides['Percentage(%)'],2)
Glycosides